<a href="https://colab.research.google.com/github/BenjaminEngel919/Prediction-of-Product-Sales/blob/main/Product_Sales_Project_1_Part_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Benjamin Engel

##For Part 5, you will go back to your original, uncleaned, sales prediction dataset with the goal of preventing data leakage.

  *You should load a fresh version of the original data set here using pd.read_csv() and start your cleaning process over to ensure there is no data leakage!*

  1. Before splitting your data, you can drop duplicates and fix inconsistencies in categorical data.* (*There is a way to do this after the split, but for this project, you may perform this step before the split)
  2. Identify the features (X) and target (y): Assign the "Item_Outlet_Sales" column as your target and the rest of the relevant variables as your features matrix.
  3. Perform a train test split
  4. Create a preprocessing object to prepare the dataset for Machine Learning
  5. Make sure your imputation of missing values occurs after the train test split using SimpleImputer.

Commit your work to GitHub.

In [1]:
## Import and mount drive from Google Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# The Usual Imports
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn import set_config
set_config(display='diagram')

In [3]:
# Import the data
path = '/content/drive/MyDrive/sales_predictions_2023.csv'
sales_predictions = pd.read_csv(path)
sales_predictions.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


#I will make a copy of original dataframe to avoid any manipulations 

In [4]:
copy_sales = sales_predictions.copy()

In [5]:
copy_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [6]:
copy_sales.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


##**Identify each feature**

- Pass through = none
- Ordinal features = None
- Numeric features = ['Item_Weight', 'Item_Visibility', 'Item_MRP]
- Nominal features = ['Item_Identifier', 'Item_Fat_Content', 'Item_Type', 'Outlet_Identifier', 'Outlet_Location_Type', 'Outlet_Size', 'Outlet_Type']
- Date/time features = ['Outlet_Establishment_Year']


##1a. Drop Duplicates

In [7]:
##3Any duplicates? If yes, drop them.
print(f'There are {copy_sales.duplicated().sum()} duplicate rows.')

There are 0 duplicate rows.


#1b. Fix Inconsistencies in Categorical Data

In [9]:
data_types = copy_sales.dtypes
str_cols = data_types[ data_types=='object'].index
str_cols

Index(['Item_Identifier', 'Item_Fat_Content', 'Item_Type', 'Outlet_Identifier',
       'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type'],
      dtype='object')

In [10]:
for col in str_cols:
  print(f'- {col}:')
  print(copy_sales[col].value_counts(dropna=False))
  print("\n\n")

- Item_Identifier:
FDW13    10
FDG33    10
NCY18     9
FDD38     9
DRE49     9
         ..
FDY43     1
FDQ60     1
FDO33     1
DRF48     1
FDC23     1
Name: Item_Identifier, Length: 1559, dtype: int64



- Item_Fat_Content:
Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64



- Item_Type:
Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
Name: Item_Type, dtype: int64



- Outlet_Identifier:
OUT027    935
OUT013    932
OUT049    930
OUT046    930
OUT035    930
OUT045    929
OUT018    928
OUT017    9

In [11]:
# Display the unique values from the column
print(copy_sales['Item_Fat_Content'].unique())

['Low Fat' 'Regular' 'low fat' 'LF' 'reg']


In [12]:
##Using a Dictionary to replace value
copy_sales.replace({'LF': 'Low Fat', 'reg': 'Regular', 'low fat': 'Low Fat'}, inplace = True)

In [13]:
## Displaying the new value totals
copy_sales['Item_Fat_Content'].value_counts()

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

In [14]:
# 'Outlet_Establishment_Year' is a date/time and will be included with categorical features
copy_sales['Outlet_Establishment_Year'] = copy_sales['Outlet_Establishment_Year'].astype("object")

##2. Define features (X) and target (y)

In [15]:
#Defining my Target variable which is y and the remaining features X
X = copy_sales.drop(columns = ['Item_Identifier','Item_Outlet_Sales'])
y = copy_sales['Item_Outlet_Sales']

In [16]:
copy_sales.dtypes

Item_Identifier               object
Item_Weight                  float64
Item_Fat_Content              object
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year     object
Outlet_Size                   object
Outlet_Location_Type          object
Outlet_Type                   object
Item_Outlet_Sales            float64
dtype: object

#3. Train Test Split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size=.30)

In [18]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5966 entries, 1921 to 7270
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Weight                4943 non-null   float64
 1   Item_Fat_Content           5966 non-null   object 
 2   Item_Visibility            5966 non-null   float64
 3   Item_Type                  5966 non-null   object 
 4   Item_MRP                   5966 non-null   float64
 5   Outlet_Identifier          5966 non-null   object 
 6   Outlet_Establishment_Year  5966 non-null   object 
 7   Outlet_Size                4276 non-null   object 
 8   Outlet_Location_Type       5966 non-null   object 
 9   Outlet_Type                5966 non-null   object 
dtypes: float64(3), object(7)
memory usage: 512.7+ KB


In [19]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2557 entries, 7503 to 3742
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Weight                2117 non-null   float64
 1   Item_Fat_Content           2557 non-null   object 
 2   Item_Visibility            2557 non-null   float64
 3   Item_Type                  2557 non-null   object 
 4   Item_MRP                   2557 non-null   float64
 5   Outlet_Identifier          2557 non-null   object 
 6   Outlet_Establishment_Year  2557 non-null   object 
 7   Outlet_Size                1837 non-null   object 
 8   Outlet_Location_Type       2557 non-null   object 
 9   Outlet_Type                2557 non-null   object 
dtypes: float64(3), object(7)
memory usage: 219.7+ KB


#4. Instantiate Column Selectors

In [20]:
#Column Selectors
cat_selector = make_column_selector(dtype_include='object')
num_selector = make_column_selector(dtype_include='number')

In [21]:
cat_selector(X_train)

['Item_Fat_Content',
 'Item_Type',
 'Outlet_Identifier',
 'Outlet_Establishment_Year',
 'Outlet_Size',
 'Outlet_Location_Type',
 'Outlet_Type']

In [22]:
num_selector(X_train)

['Item_Weight', 'Item_Visibility', 'Item_MRP']

#5a. Instantiate Transformers

In [23]:
#Imputers
freq_imputer = SimpleImputer(strategy='most_frequent')
median_imputer = SimpleImputer(strategy='median')
#Scaler
scaler = StandardScaler()
#One-hot encoder
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)

#5b. Instantiate Pipelines

In [24]:
#Numeric pipeline
numeric_pipe = make_pipeline(median_imputer, scaler)
numeric_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='median')),
                ('standardscaler', StandardScaler())])

In [25]:
#Categorical pipeline
categorical_pipe = make_pipeline(freq_imputer, ohe)
categorical_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('onehotencoder',
                 OneHotEncoder(handle_unknown='ignore', sparse=False))])

#5c. Instantiate Column Transformer

In [26]:
#Tuples for Column Transformer
number_tuple = (numeric_pipe, num_selector)
category_tuple = (categorical_pipe, cat_selector)

#ColumnTransformer
preprocessor = make_column_transformer(number_tuple, category_tuple, remainder='passthrough', verbose_feature_names_out=False)
preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fe3c760f6a0>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fe3c760e0b0>)],
                  verbose_feature_names_out=False)

#**Fit and Transform Data**

In [27]:
#Fit on Train data
preprocessor.fit(X_train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fe3c760f6a0>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fe3c760e0b0>)],
                  verbose_feature_names_out=False)

In [28]:
preprocessor.fit(X_train).get_feature_names_out()

array(['Item_Weight', 'Item_Visibility', 'Item_MRP',
       'Item_Fat_Content_Low Fat', 'Item_Fat_Content_Regular',
       'Item_Type_Baking Goods', 'Item_Type_Breads',
       'Item_Type_Breakfast', 'Item_Type_Canned', 'Item_Type_Dairy',
       'Item_Type_Frozen Foods', 'Item_Type_Fruits and Vegetables',
       'Item_Type_Hard Drinks', 'Item_Type_Health and Hygiene',
       'Item_Type_Household', 'Item_Type_Meat', 'Item_Type_Others',
       'Item_Type_Seafood', 'Item_Type_Snack Foods',
       'Item_Type_Soft Drinks', 'Item_Type_Starchy Foods',
       'Outlet_Identifier_OUT010', 'Outlet_Identifier_OUT013',
       'Outlet_Identifier_OUT017', 'Outlet_Identifier_OUT018',
       'Outlet_Identifier_OUT019', 'Outlet_Identifier_OUT027',
       'Outlet_Identifier_OUT035', 'Outlet_Identifier_OUT045',
       'Outlet_Identifier_OUT046', 'Outlet_Identifier_OUT049',
       'Outlet_Establishment_Year_1985', 'Outlet_Establishment_Year_1987',
       'Outlet_Establishment_Year_1997', 'Outlet_Establishme

In [29]:
# Transform Train and Test
X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)

In [30]:
# Display the datatype of X_train_scaled
type(X_train_processed)

numpy.ndarray

In [31]:
X_train_processed

array([[-1.22586739,  1.6066808 , -0.40189546, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.62771899, -1.00844167, -0.61928426, ...,  1.        ,
         0.        ,  0.        ],
       [-0.05644255, -0.22706823, -0.20253536, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 1.11772641, -0.91757329,  1.5257291 , ...,  1.        ,
         0.        ,  0.        ],
       [ 1.77004251, -0.22428724, -0.3811361 , ...,  1.        ,
         0.        ,  0.        ],
       [ 0.8212191 , -0.95575131, -0.73573148, ...,  1.        ,
         0.        ,  0.        ]])

#**Convert Preprocessed Array to Dataframes**

In [32]:
# Create dataframes from the processed arrays
# Default X_train and X_test
cols = preprocessor.fit(X_train).get_feature_names_out()

X_train_processed_df = pd.DataFrame(X_train_processed, 
                          columns = cols,
                          index = X_train.index)

#**Inspect the Result**

In [33]:
#Check for missing values and that data is scaled and one-hot encoded#print(np.isnan(X_train_processed).sum().sum(), 'missing values in training data')
print(np.isnan(X_train_processed).sum().sum(), 'Missing Values in Training Data.')
print('\n')
print('All Data in X_train_processed are', X_train_processed.dtype)
print('\n')
print('Shape of Data is', X_train_processed.shape)
print('\n')
#Check the data types
print(f'\nThe datatypes are {X_train_processed_df.dtypes}')
X_train_processed

0 Missing Values in Training Data.


All Data in X_train_processed are float64


Shape of Data is (5966, 50)



The datatypes are Item_Weight                        float64
Item_Visibility                    float64
Item_MRP                           float64
Item_Fat_Content_Low Fat           float64
Item_Fat_Content_Regular           float64
Item_Type_Baking Goods             float64
Item_Type_Breads                   float64
Item_Type_Breakfast                float64
Item_Type_Canned                   float64
Item_Type_Dairy                    float64
Item_Type_Frozen Foods             float64
Item_Type_Fruits and Vegetables    float64
Item_Type_Hard Drinks              float64
Item_Type_Health and Hygiene       float64
Item_Type_Household                float64
Item_Type_Meat                     float64
Item_Type_Others                   float64
Item_Type_Seafood                  float64
Item_Type_Snack Foods              float64
Item_Type_Soft Drinks              float64
Item_Type_

array([[-1.22586739,  1.6066808 , -0.40189546, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.62771899, -1.00844167, -0.61928426, ...,  1.        ,
         0.        ,  0.        ],
       [-0.05644255, -0.22706823, -0.20253536, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 1.11772641, -0.91757329,  1.5257291 , ...,  1.        ,
         0.        ,  0.        ],
       [ 1.77004251, -0.22428724, -0.3811361 , ...,  1.        ,
         0.        ,  0.        ],
       [ 0.8212191 , -0.95575131, -0.73573148, ...,  1.        ,
         0.        ,  0.        ]])

In [34]:
#View the transformations
X_train_processed_df.describe().round(2)

,Item_Weight,Item_Visibility,Item_MRP,Item_Fat_Content_Low Fat,Item_Fat_Content_Regular,Item_Type_Baking Goods,Item_Type_Breads,Item_Type_Breakfast,Item_Type_Canned,Item_Type_Dairy,...,Outlet_Size_High,Outlet_Size_Medium,Outlet_Size_Small,Outlet_Location_Type_Tier 1,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3
count,5966.00,5966.00,5966.00,5966.00,5966.00,5966.00,5966.00,5966.00,5966.00,5966.00,...,5966.00,5966.00,5966.00,5966.00,5966.00,5966.00,5966.00,5966.00,5966.00,5966.00
mean,0.00,-0.00,0.00,0.65,0.35,0.07,0.03,0.01,0.08,0.08,...,0.11,0.61,0.28,0.28,0.33,0.40,0.12,0.65,0.11,0.11
std,1.00,1.00,1.00,0.48,0.48,0.26,0.16,0.12,0.26,0.27,...,0.31,0.49,0.45,0.45,0.47,0.49,0.33,0.48,0.31,0.32
min,-1.98,-1.29,-1.76,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,-0.80,-0.76,-0.76,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
50%,-0.06,-0.23,0.03,1.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00
75%,0.76,0.56,0.72,1.00,1.00,0.00,0.00,0.00,0.00,0.00,...,0.00,1.00,1.00,1.00,1.00,1.00,0.00,1.00,0.00,0.00
max,2.01,5.14,2.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
